In [ ]:
import boto3
import sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
print("Default bucket is {}".format(bucket))

In [ ]:
### REPLACE INPUT FROM PREVIOUS SAGEMAKER PROCESSING CONFIG OUTPUT #####
prefix="customer_support_classification"
s3_training_path="https://sagemaker-us-east-1-123412341234.s3.amazonaws.com/sagemaker-scikit-learn-2020-11-16-19-27-42-281/output/training/training.txt"
s3_validation_path="https://sagemaker-us-east-1-123412341234.s3.amazonaws.com/sagemaker-scikit-learn-2020-11-16-19-27-42-281/output/validation/validation.txt"
s3_output_path="s3://{}/{}".format(bucket, prefix)

In [ ]:
from sagemaker import image_uris
region_name = boto3.Session().region_name
print("Training the model in {} region".format(region_name))

# Estimator

In [ ]:
container = image_uris.retrieve('blazingtext', region=region_name)
print("The algo container is {}".format(container))

In [ ]:
blazing_text = sagemaker.estimator.Estimator(
    container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.c4.4xlarge',
    output_path=s3_output_path
)

# set the hyperparameters

In [ ]:
blazing_text.set_hyperparameters(mode='supervised')

In [ ]:
from sagemaker import TrainingInput

In [ ]:
train_data = TrainingInput(s3_training_path, 
                           distribution='FullyReplicated',
                          content_type='text/plain',
                           s3_data_type='S3Prefix'
                          )
validation_data = TrainingInput(
    s3_validation_path,
    distribution='FullyReplicated',
    content_type='text/plain',
    s3_data_type='S3Prefix'
)

s3_channels = {'train': train_data,
               'validation': validation_data
              }
blazing_text.fit(inputs=s3_channels)

In [ ]:
blazing_text.latest_training_job.job_name

In [ ]:
blazing_text_predictor = blazing_text.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium'
)

In [ ]:
import json
import nltk

nltk.download('punkt')


In [ ]:

"""
Hi my outlook app seems to misbehave a lot lately, I cannot sync my emails and it often crashes and asks for
credentials. Could you help me out?
"""
sentences = ["Hi my outlook app seems to misbehave a lot lately, I cannot sync my emails and it often crashes and asks for credentials.", "Could you help me out?"]
tokenized_sentences = [' '.join(nltk.word_tokenize(sent)) for sent in sentences]
payload = {"instances" : tokenized_sentences,
          "configuration": {"k": 1}}

In [ ]:
payload

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

case_classifier = Predictor(
    endpoint_name="blazingtext-2020-11-18-15-13-52-229", # Replace with sagemaker endpoint deployed in the previous step
    serializer=JSONSerializer()
)
response = case_classifier.predict(payload)

predictions = json.loads(response)
print(predictions)

In [ ]:
predictions = sorted(predictions, key=lambda i: i['prob'], reverse=True)

In [ ]:
print(json.dumps(predictions[0], indent=2))